In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns
url = 'E:\App Exes\VS Code\Proyecto Final Data\healthcare-dataset-stroke-data.csv'

In [4]:
df = pd.read_csv(url, sep = ',')
df = pd.get_dummies(df, columns=['work_type'], prefix='work_type')
df = pd.get_dummies(df, columns=['smoking_status'], prefix='smoking_status')
#Reemplaza valores nulos de BMI promediando por rango edades cercanas
promedio_bmi_edades = df.groupby('age')['bmi'].mean()
df['bmi'] = df.apply(lambda row: promedio_bmi_edades[row['age']] if pd.isnull(row['bmi']) else row['bmi'], axis=1)

#Reemplazo valores strings por valores numericos
df['gender'] = df['gender'].replace({'Male': 1, 'Female': 0, 'Other': 2})
df['ever_married'] = df['ever_married'].replace({'No': 0, 'Yes': 1})
df['Residence_type'] = df['Residence_type'].replace({'Rural': 0, 'Urban': 1})

df_stroke = df.loc[df['stroke'] == 1]
df_nostroke = df.loc[df['stroke'] == 0]

#Muestro el dataframe completo
df

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,1,67.0,0,1,1,1,228.69,36.600000,1,False,False,True,False,False,False,True,False,False
1,51676,0,61.0,0,0,1,0,202.21,30.190000,1,False,False,False,True,False,False,False,True,False
2,31112,1,80.0,0,1,1,0,105.92,32.500000,1,False,False,True,False,False,False,False,True,False
3,60182,0,49.0,0,0,1,1,171.23,34.400000,1,False,False,True,False,False,False,False,False,True
4,1665,0,79.0,1,0,1,0,174.12,24.000000,1,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,0,80.0,1,0,1,1,83.75,28.783582,0,False,False,True,False,False,False,False,True,False
5106,44873,0,81.0,0,0,1,1,125.20,40.000000,0,False,False,False,True,False,False,False,True,False
5107,19723,0,35.0,0,0,1,0,82.99,30.600000,0,False,False,False,True,False,False,False,True,False
5108,37544,1,51.0,0,0,1,0,166.29,25.600000,0,False,False,True,False,False,False,True,False,False


¿Podemos predecir los ACV antes de que ocurran en un paciente?

En este caso como el objetivo del modelo es predecir el porcentaje de posibilidades de que un paciente sufra un ACV, voy a usar un algoritmo de regresion lineal ya que la variable que quiero predecir es continua.

Divido el dataframe en casos mas particulares df_train son casos de no ACV que no tienen ni hipertension ni DVC y casos de ACV con hipertension y/o DCV df_test son los casos restantes en el cual hay casos con ACV sin hipertension ni DVC y casos sin ACV con hipertension y/o DVC

In [5]:
# Seleccionar pacientes que no tuvieron ACV y no tienen hipertensión ni DVC
cond1 = (df['stroke'] == 0) & (df['hypertension'] == 0) & (df['heart_disease'] == 0)
df1 = df[cond1]

# Seleccionar pacientes que tuvieron ACV y tienen hipertensión o DVC
cond2 = (df['stroke'] == 1) & ((df['hypertension'] == 1) | (df['heart_disease'] == 1))
df2 = df[cond2]

# Concatenar los dos dataframes anteriores
df_train = pd.concat([df1, df2])

# Seleccionar pacientes que no tuvieron ACV pero tienen hipertensión o DVC, y aquellos que no cumplen ninguna de las condiciones anteriores
cond3 = ~(cond1 | cond2)
df_test = df[cond3]

In [6]:
# Seleccionar las características y la variable objetivo para df
X_train = df_train[['gender', 'age', 'hypertension', 'heart_disease', 'ever_married','Residence_type', 'avg_glucose_level', 'bmi', 'work_type_Govt_job', 'work_type_Never_worked', 'work_type_Private', 'work_type_Self-employed', 'work_type_children', 'smoking_status_Unknown', 'smoking_status_formerly smoked',	'smoking_status_never smoked', 'smoking_status_smokes']]
y_train = df_train['stroke']

# Dividir df en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=83)

# Crear el modelo de regresión lineal
reg = LogisticRegression()

# Entrenar el modelo con los datos de entrenamiento de df
reg.fit(X_train, y_train)

# Predecir los valores de stroke para los datos de entrenamiento y prueba de df
train_pred = reg.predict(X_train)
test_pred = reg.predict(X_test)

# Calcular la precisión para los datos de entrenamiento y prueba de df
train_acc = r2_score(y_train, train_pred)
test_acc = r2_score(y_test, test_pred)

# Imprimir los resultados
print("Precisión en datos de entrenamiento:", train_acc)
print("Precisión en datos de prueba:", test_acc)

Precisión en datos de entrenamiento: 1.0
Precisión en datos de prueba: 1.0


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Procedo a probar el modelo con casos reales del dataset para comprobar si los resultados son los mismos

In [7]:
# Predecir para datos de entrenamiento
train_pred = reg.predict(X_train)
train_acc = accuracy_score(y_train, np.round(train_pred))

# Predecir para datos de prueba
test_pred = reg.predict(X_test)
test_acc = accuracy_score(y_test, np.round(test_pred))

print("Precisión en datos de entrenamiento:", train_acc)
print("Precisión en datos de prueba:", test_acc)

Precisión en datos de entrenamiento: 1.0
Precisión en datos de prueba: 1.0


In [8]:
# Definir el ID del paciente que deseas mostrar
id_paciente = 19723

# Filtrar el DataFrame para obtener los detalles del paciente con el ID dado
paciente_seleccionado = df[df['id'] == id_paciente]

# Mostrar los detalles del paciente seleccionado
paciente_seleccionado

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
5107,19723,0,35.0,0,0,1,0,82.99,30.6,0,False,False,False,True,False,False,False,True,False


In [9]:
# Este caso en particular es del df_train
# id: 19723, ACV = 0

# Definir el ID del paciente para el cual deseas hacer la predicción
id_paciente = 19723

# Filtrar el DataFrame por el ID del paciente y seleccionar todas las columnas excepto "stroke"
datos_paciente = df[df['id'] == id_paciente].drop(columns=['id', 'stroke'])

# Convertir los datos del paciente en un array numpy para la predicción
new_data = np.array(datos_paciente)

# Obtener la probabilidad del modelo para las características del paciente
probabilities = reg.predict_proba(new_data)

# La probabilidad de tener un ACV corresponde a la clase 1 (índice 1) del arreglo probabilities
probability_acv = probabilities[0, 1]

# Convertir la probabilidad en un valor entre 0 y 100
probability_percentage = probability_acv * 100

print("La probabilidad de tener un ACV es:", probability_percentage)

La probabilidad de tener un ACV es: 0.154669861007205


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [10]:
# Definir el ID del paciente que deseas mostrar
id_paciente = 31112

# Filtrar el DataFrame para obtener los detalles del paciente con el ID dado
paciente_seleccionado = df[df['id'] == id_paciente]

# Mostrar los detalles del paciente seleccionado
paciente_seleccionado

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
2,31112,1,80.0,0,1,1,0,105.92,32.5,1,False,False,True,False,False,False,False,True,False


In [11]:
# Este caso en particular es del df_train
# id: 31112, ACV = 1

# Definir el ID del paciente para el cual deseas hacer la predicción
id_paciente = 31112

# Filtrar el DataFrame por el ID del paciente y seleccionar todas las columnas excepto "stroke"
datos_paciente = df[df['id'] == id_paciente].drop(columns=['id', 'stroke'])

# Convertir los datos del paciente en un array numpy para la predicción
new_data = np.array(datos_paciente)

# Obtener la probabilidad del modelo para las características del paciente
probabilities = reg.predict_proba(new_data)

# La probabilidad de tener un ACV corresponde a la clase 1 (índice 1) del arreglo probabilities
probability_acv = probabilities[0, 1]

# Convertir la probabilidad en un valor entre 0 y 100
probability_percentage = probability_acv * 100

print("La probabilidad de tener un ACV es:", probability_percentage)

La probabilidad de tener un ACV es: 86.11384413522983


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [12]:
# Definir el ID del paciente que deseas mostrar
id_paciente = 56669

# Filtrar el DataFrame para obtener los detalles del paciente con el ID dado
paciente_seleccionado = df[df['id'] == id_paciente]

# Mostrar los detalles del paciente seleccionado
paciente_seleccionado

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
5,56669,1,81.0,0,0,1,1,186.21,29.0,1,False,False,True,False,False,False,True,False,False


In [13]:
# Este caso en particular es del df_test
# id: 56669, ACV = 1

# Definir el ID del paciente para el cual deseas hacer la predicción
id_paciente = 56669

# Filtrar el DataFrame por el ID del paciente y seleccionar todas las columnas excepto "stroke"
datos_paciente = df[df['id'] == id_paciente].drop(columns=['id', 'stroke'])

# Convertir los datos del paciente en un array numpy para la predicción
new_data = np.array(datos_paciente)

# Obtener la probabilidad del modelo para las características del paciente
probabilities = reg.predict_proba(new_data)

# La probabilidad de tener un ACV corresponde a la clase 1 (índice 1) del arreglo probabilities
probability_acv = probabilities[0, 1]

# Convertir la probabilidad en un valor entre 0 y 100
probability_percentage = probability_acv * 100

print("La probabilidad de tener un ACV es:", probability_percentage)

La probabilidad de tener un ACV es: 1.8296381028315651


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [14]:
# Definir el ID del paciente que deseas mostrar
id_paciente = 6369

# Filtrar el DataFrame para obtener los detalles del paciente con el ID dado
paciente_seleccionado = df[df['id'] == id_paciente]

# Mostrar los detalles del paciente seleccionado
paciente_seleccionado

,id,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
5091,6369,1,59.0,1,0,1,0,95.05,30.9,0,False,False,True,False,False,False,False,True,False


In [17]:
# Este caso en particular es del df_test
# id: 6369, ACV = 0

# Definir el ID del paciente para el cual deseas hacer la predicción
id_paciente = 6369

# Filtrar el DataFrame por el ID del paciente y seleccionar todas las columnas excepto "stroke"
datos_paciente = df[df['id'] == id_paciente].drop(columns=['id', 'stroke'])

# Convertir los datos del paciente en un array numpy para la predicción
new_data = np.array(datos_paciente)

# Obtener la probabilidad del modelo para las características del paciente
probabilities = reg.predict_proba(new_data)

# La probabilidad de tener un ACV corresponde a la clase 1 (índice 1) del arreglo probabilities
probability_acv = probabilities[0, 1]

# Convertir la probabilidad en un valor entre 0 y 100
probability_percentage = probability_acv * 100

print("La probabilidad de tener un ACV es:", probability_percentage)


La probabilidad de tener un ACV es: 80.6472931651647


C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Conclusion final sobre el modelo: El modelo se entreno con casos que fueran claros, es decir, si tienen hipertension o enfermedades del corazon y sufrieron ACV o no tienen hipertension ni enfermedades del corazon y no sufrieron ACV, ya que hay casos que si tienen hipertension o enfermedades del corazon y no sufrieron ACV, pero esto no significa que las posibilidades de que lo sufra, al contrario seria mal informar al modelo, tambien hay casos que sufrieron ACV pero no tienen hipertension o enfermedades del corazon estos casos harian que el modelo le quitara peso a estas variables importantes, igual estos casos se usaron para probar al modelo y fueron correctas las predicciones, algunas como la ultima el paciente 6369 que dio altas posibilidades pero el paciente no sufrio ACV, personalmente creo que la prediccion es correcta ya que el paciente sufre hipertencion, esta casado, es empleado y tiene 59 años. Estas variables junto con el estudio de insight que hice son valores que dan a entender que va a sufrir un ACV.